<a href="https://colab.research.google.com/github/2003UJAN/2003UJAN/blob/main/Laryngeal_Cancer_XAI_DenseNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch_metric_learning

In [ ]:
!pip install grad-cam

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from pytorch_metric_learning import losses
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
import matplotlib.pyplot as plt
from torchvision.models import densenet121
import zipfile
import os

In [ ]:
zip_path = 'archive.zip'
extract_path = '.'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:
dataset_path = os.path.join(extract_path, 'laryngeal dataset', 'laryngeal dataset')

In [ ]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((100, 100)),
    transforms.ToTensor()
])

In [ ]:
ef load_dataset(root_path, transform):
    return datasets.ImageFolder(root=root_path, transform=transform)

In [ ]:
fold1_path = os.path.join(dataset_path, 'FOLD 1')
fold2_path = os.path.join(dataset_path, 'FOLD 2')
fold3_path = os.path.join(dataset_path, 'FOLD 3')

In [ ]:
train_data = load_dataset(fold1_path, transform)
val_data = load_dataset(fold2_path, transform)
test_data = load_dataset(fold3_path, transform)

In [ ]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [ ]:
class DenseNetPrototypicalNetwork(nn.Module):
    def __init__(self, embedding_dim, num_classes):
        super(DenseNetPrototypicalNetwork, self).__init__()
        self.densenet = densenet121(pretrained=True)
        self.densenet.classifier = nn.Linear(self.densenet.classifier.in_features, embedding_dim)
        self.classifier = nn.Linear(embedding_dim, num_classes)

    def forward(self, x):
        features = self.densenet.features(x)
        features = torch.flatten(features, start_dim=1)
        embeddings = self.densenet.classifier(features)
        logits = self.classifier(embeddings)
        return logits

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
embedding_dim = 64
num_classes = len(train_data.classes)
model = DenseNetPrototypicalNetwork(embedding_dim, num_classes).to(device)
optimizer = optim.AdamW(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.7)
criterion = nn.CrossEntropyLoss()

In [ ]:
def compute_accuracy(loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            logits = model(images)
            predictions = torch.argmax(logits, dim=1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
    return correct / total

for epoch in range(40):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        logits = model(images)
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    scheduler.step()
    train_acc = compute_accuracy(train_loader)
    val_acc = compute_accuracy(val_loader)

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}, '
          f'Train Accuracy: {train_acc*100:.2f}%, Val Accuracy: {val_acc*100:.2f}%')

test_acc = compute_accuracy(test_loader)
print(f'Test Accuracy: {test_acc*100:.2f}%')

In [ ]:
def generate_gradcam(model, image, target_layer):
    model.eval()
    if image.shape[0] != model.densenet.features[0].weight.shape[1]:
        image = image.repeat(model.densenet.features[0].weight.shape[1], 1, 1)
    cam = GradCAM(model=model, target_layers=[target_layer])
    grayscale_cam = cam(input_tensor=image.unsqueeze(0))[0, :]
    image = image.permute(1, 2, 0).cpu().numpy()
    cam_image = show_cam_on_image(image, grayscale_cam, use_rgb=True)
    return cam_image

In [ ]:
test_iter = iter(test_loader)
images, labels = next(test_iter)
image = images[0].to(device)
target_layer = model.densenet.features[-1]
cam_image = generate_gradcam(model, image, target_layer

In [ ]:
plt.imshow(cam_image)
plt.title('Grad-CAM')
plt.show()